In [ ]:
%reload_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
import os, sys, inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0, current_dir)
import torch
import torch.optim as optim
import numpy as np
import argparse
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tensorboardX import SummaryWriter
from models.model_RCNN_only import RCNN_only
from maskrcnn_rui.data.transforms import build_transforms_maskrcnn
from maskrcnn_rui.config import cfg
import utils.model_utils as model_utils
from utils.logger import setup_logger, printer
from maskrcnn_benchmark.utils.comm import synchronize, get_rank
from utils.checkpointer import DetectronCheckpointer
from utils.utils_misc import colored

parser = argparse.ArgumentParser(description="Rui's Scale Estimation Network Training")
# Training
parser.add_argument('--task_name', type=str, default='tmp', help='resume training')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=8)
parser.add_argument('--batchsize', type=int, default=36, help='input batch size during training')
parser.add_argument('--save_every_iter', type=int, default=0, help='set to 0 to save ONLY at the end of each epoch')
# parser.add_argument('--batchsizeeval', type=int, default=42, help='input batch size during evaluation')
parser.add_argument('--niter', type=int, default=5000, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate, default=0.005')
parser.add_argument('--beta1', type=float, default=0.9, help='beta1 for adam. default=0.5')
parser.add_argument('--not_val', action='store_true', help='Do not validate duruign training')
parser.add_argument('--save_every_epoch', type=int, default=10, help='save checkpoint every ? epoch')
# Model
parser.add_argument('--accu_model', action='store_true', help='Use accurate model with theta instead of Derek\'s approx.')
# Pretraining
parser.add_argument('--resume', type=str, help='resume training; can be full path (e.g. tmp/checkpoint0.pth.tar) or taskname (e.g. tmp)', default='NoCkpt')
parser.add_argument('--feature_only', action='store_true', help='restore only features (remove all classifiers) from checkpoint')
# Device
parser.add_argument('--cpu', action='store_true', help='Force training on CPU')
parser.add_argument("--local_rank", type=int, default=0)
parser.add_argument("--master_port", type=str, default='8914')
# DEBUG
parser.add_argument('--debug', action='store_true', help='Debug eval')
# Mask R-CNN
parser.add_argument('--not_rcnn', action='store_true', help='Disable Mask R-CNN module')

parser.add_argument('--pointnet_camH', action='store_true', help='')
parser.add_argument('--est_bbox', action='store_true', help='Enable estimating bboxes instead of using GT bboxes')

parser.add_argument(
    "--config-file",
    default="",
    metavar="FILE",
    help="path to config file",
    type=str,
)
parser.add_argument(
    "opts",
    help="Modify config options using the command-line",
    default=None,
    nargs=argparse.REMAINDER,
)

# opt = parser.parse_args()
opt = parser.parse_args('--batchsize=1 --task_name tmp_eval --niter 1 --accu_model --resume YES \
--config-file coco_config_small_RCNNOnly.yaml \
SOLVER.IMS_PER_BATCH 1 TEST.IMS_PER_BATCH 1'.split())

opt.checkpoints_folder = 'checkpoint'

# config_file = "maskrcnn/coco_config.yaml"
config_file = opt.config_file
cfg.merge_from_file(config_file)
# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])
cfg.merge_from_list(opt.opts)
cfg.freeze()
opt.cfg = cfg

# === DISTRIBUTED TRAINING
num_gpus = int(os.environ["WORLD_SIZE"]) if "WORLD_SIZE" in os.environ else 1
opt.distributed = num_gpus > 1
if opt.distributed:
    torch.cuda.set_device(opt.local_rank)
    torch.distributed.init_process_group(
        backend="nccl", init_method="env://"
    )
    synchronize()
# device = torch.device("cuda" if torch.cuda.is_available() and not opt.cpu else "cpu")
device = 'cuda'
print("Device:", device)
rank = get_rank()

# === SUMMARY WRITERS
summary_path = './summary/'+opt.task_name
writer = SummaryWriter(summary_path)

# === LOGGING
# sys.stdout = Logger(summary_path+'/log.txt')
logger = setup_logger("logger:train", summary_path, get_rank(), filename="logger_maskrcn-style.txt")
logger.info(colored("==[config]== opt", 'white', 'on_blue'))
logger.info(opt)
logger.info(colored("==[config]== cfg", 'white', 'on_blue'))
logger.info(cfg)
logger.info(colored("==[config]== Loaded configuration file {}".format(opt.config_file), 'white', 'on_blue'))
with open(opt.config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
printer = printer(get_rank(), debug=opt.debug)

In [ ]:
resume_task_name = '1109-0141-mm1_SUN360RCNN-HorizonPitchRollVfovNET_myDistNarrowerLarge1105_bs16on4_le1e-5_indeptClsHeads_synBNApex_valBS1_yannickTransformAug'

model = RCNN_only(cfg, opt, logger, printer, rank=rank)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=cfg.SOLVER.BASE_LR, betas=(opt.beta1, 0.999), eps=1e-5)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=20, cooldown=10)

save_to_disk = get_rank() == 0
opt.checkpoints_folder = 'checkpoint'
checkpointer = DetectronCheckpointer(
    opt, model, optimizer, scheduler, opt.checkpoints_folder, os.path.join(opt.checkpoints_folder, resume_task_name), save_to_disk, logger=logger
)
checkpoint_restored, _, _ = checkpointer.load(task_name=resume_task_name)

# === DATASET
train_trnfs_maskrcnn = build_transforms_maskrcnn(cfg, True)
eval_trnfs_maskrcnn = build_transforms_maskrcnn(cfg, False)

# Test on single image

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
from PIL import Image
from dataset_cvpr import bins2roll, bins2vfov, bins2horizon, bins2pitch
from panorama_cropping_dataset_generation.debugging import showHorizonLine

In [ ]:
model.eval() #很关键！

test_image_path = 'demo/fall-cmu-700x700.jpg'
# test_image_path = 'demo/white-house.jpg'
im_ori_RGB = Image.open(test_image_path).convert('RGB') #im_ori_RGB.size: [W, H]
im = eval_trnfs_maskrcnn(im_ori_RGB)
        
H_num, W_num = im_ori_RGB.size
list_of_oneLargeBbox_list_cpu = model_utils.oneLargeBboxList([W_num], [H_num])
list_of_oneLargeBbox_list = [bbox_list_array.to(device) for bbox_list_array in list_of_oneLargeBbox_list_cpu]

output_RCNN = model(image_batch_list=[im.to(device)], list_of_oneLargeBbox_list=list_of_oneLargeBbox_list)
output_horizon = output_RCNN['output_horizon']
output_pitch = output_RCNN['output_pitch']
output_roll = output_RCNN['output_roll']
output_vfov = output_RCNN['output_vfov']

idx = 0

im = im_ori_RGB
if len(im.getbands()) == 1:
    im = Image.fromarray(np.tile(np.asarray(im)[:,:,np.newaxis], (1, 1, 3)))

horizon_disc = output_horizon[idx].detach().cpu().numpy().squeeze()
pitch_disc = output_pitch[idx].detach().cpu().numpy().squeeze()
roll_disc = output_roll[idx].detach().cpu().numpy().squeeze()
vfov_disc = output_vfov[idx].detach().cpu().numpy().squeeze()
vfov_disc[...,0] = -35
vfov_disc[...,-1] = -35

horizon = bins2horizon(horizon_disc)
pitch = bins2pitch(pitch_disc)
roll = bins2roll(roll_disc)
vfov = bins2vfov(vfov_disc)
w, h = im.size
f_pix = h / 2. / np.tan(vfov / 2.)
# sensor_size = sensor_size_num[idx]
sensor_size = 24 # !!!!!!
f_mm = f_pix / h * sensor_size

im2, _ = showHorizonLine(np.asarray(im).copy(), vfov, pitch, roll, focal_length=f_mm, debug=True, color=(0, 0, 255), width=3) # Blue: horizon converted from camera params with roll

plt.figure(figsize=(15, 15))
plt.imshow(im2)
plt.show()
plt.close()

In [ ]:
im.size